In [ ]:
#==================================================================
#EXTERNAL SLADS SETUP
#==================================================================

#==================================================================
#GENERAL LIBRARY IMPORTS
#==================================================================
from __future__ import absolute_import, division, print_function
import cv2
import contextlib
import copy
import datetime
import gc
import glob
import logging
import math
import matplotlib
import matplotlib.pyplot as plt
import multiplierz
import multiprocessing
import natsort
import numpy as np
import numpy.matlib as matlib
import os
import pandas as pd
import pickle
import PIL
import PIL.ImageOps
import platform
import ray
import re
import sys
import scipy
import shutil
import sklearn
import time
import warnings

from IPython import display
from IPython.core.debugger import Tracer
from itertools import chain
from joblib import Parallel, delayed
from matplotlib.pyplot import figure
from multiplierz.mzAPI import mzFile
from multiplierz.spectral_process import mz_range
from numba import jit
from PIL import Image
from scipy import misc
from scipy import signal
from scipy.io import loadmat
from scipy.io import savemat
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks
from scipy.special import softmax
from sklearn import linear_model
from sklearn import svm
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPRegressor as nnr
from sklearn.preprocessing import *
from sklearn.utils import shuffle
from skimage.util import view_as_windows as viewW
from skimage import filters
from skimage.filters import *
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity
from skimage.transform import resize
from sobol import *
from tqdm.auto import tqdm

import skimage
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.datasets import load_digits

ray.init()

In [ ]:
lineExt = '.raw'
window = 20*1e-6

#Obtain sample directories
sampleFolders = natsort.natsorted(glob.glob('./INPUTS-ut/*'), reverse=False)

sampleFolder = sampleFolders[0]
#for sampleFolder in sampleFolders:

sampleName = sampleFolder.split(os.path.sep)[-1]
#sampleNames.append(sampleName)
print(sampleName)

#Read in data from sampleInfo.txt
lineIndex = 0
sampleInfo = open(sampleFolder+os.path.sep+'sampleInfo.txt').readlines()

#Read the max number of lines that are expected 
numLines = int(sampleInfo[lineIndex].rstrip())
lineIndex += 1

#Read the sample width
sampleWidth = float(sampleInfo[lineIndex].rstrip())
lineIndex += 1

#Read the sample height
sampleHeight = float(sampleInfo[lineIndex].rstrip())
lineIndex += 1

#Read in scan rate
scanRate = float(sampleInfo[lineIndex].rstrip())
lineIndex += 1

finalWidth = int(round((sampleWidth*1e3)/scanRate))
newTimes = np.linspace(0, (finalWidth-1)/60, finalWidth)

#Get paths for raw data files
scanFiles = natsort.natsorted(glob.glob(sampleFolder+os.path.sep+'*'+lineExt), reverse=False)

#Get original timing information
dataFiles = [mzFile(scanFileName) for scanFileName in scanFiles]
origTimes = [np.asarray(data.xic(data.time_range()[0], data.time_range()[1]))[:,0] for data in dataFiles]

data = dataFiles[int(len(dataFiles)/2)]
data = np.asarray(data.scan(int(len(data.scan_info())/2)))
np.savetxt('./mz219-Example.csv', data, delimiter=',')
mzValue = 219.02664
lowMZ, highMZ = mzValue*(1-window), mzValue*(1+window)
origImage = [np.asarray(data.xic(data.time_range()[0], data.time_range()[1], lowMZ, highMZ))[:,1] for data in dataFiles]
normImage = np.asarray([np.interp(newTimes, origTimes[lineNum], origImage[lineNum]) for lineNum in range(0, len(origImage))])
plt.imshow(normImage, aspect='auto', cmap='hot')
plt.show()

fig=plt.figure()
ax=fig.add_subplot(1,1,1)
plt.axis('off')
plt.imshow(normImage, cmap='hot', aspect='auto')
extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
plt.savefig('./mz219-ExampleImage.png', bbox_inches=extent)
plt.close()













In [ ]:
lineExt = '.raw'
window = 20*1e-6

#Obtain sample directories
sampleFolders = natsort.natsorted(glob.glob('./INPUTS-ut/*'), reverse=False)

sampleFolder = sampleFolders[-6]
#for sampleFolder in sampleFolders:

sampleName = sampleFolder.split(os.path.sep)[-1]
#sampleNames.append(sampleName)
print(sampleName)

#Read in data from sampleInfo.txt
lineIndex = 0
sampleInfo = open(sampleFolder+os.path.sep+'sampleInfo.txt').readlines()

#Read the max number of lines that are expected 
numLines = int(sampleInfo[lineIndex].rstrip())
lineIndex += 1

#Read the sample width
sampleWidth = float(sampleInfo[lineIndex].rstrip())
lineIndex += 1

#Read the sample height
sampleHeight = float(sampleInfo[lineIndex].rstrip())
lineIndex += 1

#Read in scan rate
scanRate = float(sampleInfo[lineIndex].rstrip())
lineIndex += 1

finalWidth = int(round((sampleWidth*1e3)/scanRate))
newTimes = np.linspace(0, (finalWidth-1)/60, finalWidth)

#Get paths for raw data files
scanFiles = natsort.natsorted(glob.glob(sampleFolder+os.path.sep+'*'+lineExt), reverse=False)

#Get original timing information
dataFiles = [mzFile(scanFileName) for scanFileName in scanFiles]
origTimes = [np.asarray(data.xic(data.time_range()[0], data.time_range()[1]))[:,0] for data in dataFiles]

mzValue = 219.02664
lowMZ, highMZ = mzValue*(1-window), mzValue*(1+window)
origImage = [np.asarray(data.xic(data.time_range()[0], data.time_range()[1], lowMZ, highMZ))[:,1] for data in dataFiles]

maxLength = np.max([len(origLine) for origLine in origImage])


plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "10"


padded = [np.pad(origLine, (0,maxLength-len(origLine))) for origLine in origImage]
plt.imshow(padded, aspect='auto', cmap='hot')
plt.savefig('./mz219-misAligned.png')
plt.show()
plt.close()

origNormImage = np.asarray([np.interp(np.linspace(np.concatenate(origTimes).min(), np.concatenate(origTimes).max(), 1000), origTimes[lineNum], origImage[lineNum]) for lineNum in range(0, len(origImage))])
plt.imshow(origNormImage, aspect='auto', cmap='hot')
plt.savefig('./mz219-1000Norm.png')
plt.show()
plt.close()

normImage = np.asarray([np.interp(newTimes, origTimes[lineNum], origImage[lineNum]) for lineNum in range(0, len(origImage))])
plt.imshow(normImage, aspect='auto', cmap='hot')
plt.savefig('./mz219-finalNorm.png')
plt.show()
plt.close()
